# First Steps

## Creating a simple Kafka consumer app

In [ ]:
# | hide

from IPython.display import Markdown as md

from fastkafka.testing import run_script_and_cancel

In [ ]:
# | hide
# | notest

import nest_asyncio

In [ ]:
# | hide
# | notest
nest_asyncio.apply()

For our first demo we will create the simplest possible Kafka consumer and run it using uvicorn.

The consumer will:

1. Connect to the Kafka Broker we setup in the Intro guide

2. Listen to the hello topic

3. Write any message received from the hello topic to stdout
    
To create the consumer, first, create a file named <b>hello_kafka_consumer.py</b> and copy the following code to it:

In [ ]:
# | echo: false

consumer_script = """
from os import environ

from fastkafka.application import FastKafka
from pydantic import BaseModel, Field

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_app = FastKafka(
    **kafka_config,
)
    
@kafka_app.consumes()
async def on_hello(msg: HelloKafkaMsg):
    print(f"Got data, msg={msg.msg}")
"""

md(f"```python\n{consumer_script}\n```")

```python

from os import environ

from fastkafka.application import FastKafka
from pydantic import BaseModel, Field

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_app = FastKafka(
    **kafka_config,
)
    
@kafka_app.consumes()
async def on_hello(msg: HelloKafkaMsg):
    print(f"Got data, msg={msg.msg}")

```

!!! info \"Kafka configuration\"

    This consumer script uses KAFKA_HOSTNAME and KAFKA_PORT environment vars, so make sure that you have exported them into your environment before running the following comand (e.g. in shell, for KAFKA_HOSTNAME, run: 'export KAFKA_HOSTNAME=kafka').

To run this consumer, in your terminal, run:

In [ ]:
# | echo: false

consumer_cmd = "python3 -m fastkafka run hello_kafka_consumer:app"

md(f"```shell\n{consumer_cmd}\n```")

```shell
fastkafka run hello_kafka_consumer:app
```

After running the command, you should see something similar to the ouput below:

In [ ]:
# | echo: false

exit_code, output = await run_script_and_cancel(
    script=consumer_script,
    script_file="hello_kafka_consumer.py",
    cmd=consumer_cmd,
    cancel_after=30,
)

assert exit_code == 0, output.decode("utf-8")
print(output.decode("utf-8"))

FileNotFoundError: [Errno 2] No such file or directory: 'fastkafka'

Now you can interact with your consumer, by sending the messages to the subscribed 'hello' topic, don't worry, we will cover this in the next step of this guide.

## Sending first message to your consumer

After we have created and run our first consumer, we should send a message to it, to make sure it is working properly.

If you are using the Kafka setup as described in the Intro guide, you can follow the steps listed here to send a message to the hello topic.

First, connect to your running kafka broker by running:

``` shell
docker run -it kafka /bin/bash
```

Then, when connected to the container, run:

``` shell
kafka-console-producer.sh --bootstrap-server=localhost:9092 --topic=hello
```

This will open an interactive connection to the hello topic, now you can write your mesages to the topic and they will be consumed by our consumer.

In the shell, type:
``` shell
{"msg":"hello"}
```
and press enter. This will send a hello message to the topic which will be read by our running consumer and outputed to stdout.

Check the output of your consumer (terminal where you run the uvicorn command) and confirm that your consumer has read the Kafka message. You shoud see something like this:
``` shell
Got data, msg=hello
```

## Creating a hello Kafka producer

Consuming messages is only a part of this Library functionality, the other big part is producing the messages. So, let's create our first kafka producer which will send it's greetings to our consumer periodically.

The producer will:

1. Connect to the Kafka Broker we setup in the Intro guide
2. Connect to the hello topic
3. Periodically send a message to the hello world topic
    
To create the producer, first, create a file named <b>hello_kafka_producer.py</b> and copy the following code to it:

In [ ]:
# | echo: false

producer_script = """
from os import environ

import asyncio
from pydantic import BaseModel, Field

from fastkafka.application import FastKafka
from fastkafka._components.logger import get_logger

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}"
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

kafka_app = FastKafka(
    **kafka_config,
)

logger = get_logger(__name__)

@kafka_app.produces()
async def to_hello(msg: HelloKafkaMsg) -> HelloKafkaMsg:
    logger.info(f"Producing: {msg}")
    return msg

@kafka_app.run_in_background()
async def hello_every_second():
    while(True):
        await to_hello(HelloKafkaMsg(msg="hello"))
        await asyncio.sleep(1)
"""

md(f"```python\n{producer_script}\n```")

```python

from os import environ

from fastapi import FastAPI

import asyncio
from pydantic import BaseModel, Field

from fast_kafka_api.application import FastKafka
from fast_kafka_api._components.logger import get_logger

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}"
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

app = FastAPI()
kafka_app = FastKafka(
    fast_api_app=app,
    **kafka_config,
)

@app.get("/hello")
async def hello() -> str:
    return "hello"

logger = get_logger(__name__)

@kafka_app.produces()
async def to_hello(msg: HelloKafkaMsg) -> HelloKafkaMsg:
    logger.info(f"Producing: {msg}")
    return msg

@kafka_app.run_in_background()
async def hello_every_second():
    while(True):
        await to_hello(HelloKafkaMsg(msg="hello"))
        await asyncio.sleep(1)

```

!!! info \"Kafka configuration\"

    This producer script uses KAFKA_HOSTNAME and KAFKA_PORT environment vars, so make sure that you have exported them into your environment before running the following comand (e.g. in shell, for KAFKA_HOSTNAME, run: 'export KAFKA_HOSTNAME=kafka').

To run this producer, in your terminal, run:

In [ ]:
# | echo: false

producer_cmd = "python3 -m uvicorn hello_kafka_producer:app --host 0.0.0.0 --port 6006"

md(f"```shell\n{producer_cmd}\n```")

```shell
python3 -m uvicorn hello_kafka_producer:app --host 0.0.0.0 --port 6006
```

After running the command, you should see something similar to the ouput below:

In [ ]:
# | echo: false

exit_code, output = await run_script_and_cancel(
    script=producer_script,
    script_file="hello_kafka_producer.py",
    cmd=producer_cmd,
    cancel_after=30,
)

assert exit_code == 0, output.decode("utf-8")
print(output.decode("utf-8"))

INFO:     Started server process [11116]
INFO:     Waiting for application startup.
[INFO] fast_kafka_api._components.asyncapi: Old async specifications at '/tmp/tmpshq9xpb9/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] fast_kafka_api._components.asyncapi: New async specifications generated at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpshq9xpb9/asyncapi/docs.


[INFO] fast_kafka_api.application: _create_producer() : created producer using the config: '{'bootstrap_servers': 'davor-fast-kafka-api-kafka-1:9092'}'
[INFO] hello_kafka_producer: Producing: msg='hello'
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:6006 (Press CTRL+C to quit)
[INFO] hel

Now, while the producer is running, it will send a HelloKafkaMsg every second to the hello kafka topic.
If your consumer is still running, you should see the messages appear in its log.

## Recap

In this guide we have:
    
1. Created a simple Kafka consumer using FastKafka
2. Sent a message to our consumer trough Kafka
3. Created a simple Kafka producer using FastKafka